In [287]:
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
import numpy as np
import pandas as pd
import cvxpy as cp
import torch
from copy import copy, deepcopy


In [288]:
np.random.seed(42)

In [289]:
n=10

In [290]:
ds = []

In [291]:
a_min = 0.8
a_var = 0.2

b_min = 0.1
b_var = 0.2

r_min = 0.1
r_var = 1.9

x_0_min = 15.0
x_0_var = 10.0

x_ref_min = 21.0
x_ref_var = 2.0

In [292]:
for i in range(1000):
	
	a=a_var*np.random.rand() + a_min
	b=b_var*np.random.rand() + b_min
	r = r_var*np.random.rand() + r_min
	x_0=  x_0_var*np.random.rand() + x_0_min
	x_ref = x_ref_var*np.array([np.random.rand()]*(n)) + x_ref_min
	

	L = r * np.identity(n)
	I = np.identity(n)
	A = np.array([a**(i+1) for i in range(n)])
	B = np.array([[a**(i-1) * b for i in range(j, 0, -1)] + [0]*(n-j) for j in range(1, n+1)])

	u = cp.Variable(n)
	constraints = [u <= np.ones(n)*5.0, u >= np.zeros(n), cp.abs(u[1:] - u[:-1]) <= np.ones(n-1)]
	prob = cp.Problem(cp.Minimize(cp.quad_form(A*x_0 +B@u - x_ref, I) + cp.quad_form(u, L)),[u <= np.ones(n)*5.0, u >= np.zeros(n)])
	prob.solve()
	if prob.status == 'optimal': 
		#X = np.array([2*(a - a_min)/a_var -1, 2*(b - b_min)/b_var - 1, 2*(r - r_min)/r_var - 1, 2*(x_0-x_0_min)/x_0_var-1, 2*(x_ref[0]-x_ref_min)/x_ref_var -1])
		X = np.array([a, b, r, x_0, x_ref[0]])
		ds.append([X, u.value])
	

In [293]:
u.value

array([1.08678672, 1.21841839, 1.29729945, 1.32517335, 1.30265614,
       1.22925016, 1.10333299, 0.92212159, 0.68161084, 0.37648494])

In [294]:
cp.abs(u[1:] - u[:-1]).value

array([0.13163166, 0.07888106, 0.0278739 , 0.0225172 , 0.07340598,
       0.12591717, 0.1812114 , 0.24051076, 0.3051259 ])

In [295]:
[sol[1] for sol in ds]

[array([4.99426944, 5.        , 5.        , 5.        , 5.        ,
        5.        , 4.68910898, 3.99785046, 3.04308044, 1.74609936]),
 array([3.28520753, 3.59968735, 3.7774118 , 3.82599921, 3.74753234,
        3.53864762, 3.190391  , 2.68783406, 2.00943418, 1.12611103]),
 array([5.        , 5.        , 5.        , 5.        , 5.        ,
        5.        , 5.        , 5.        , 4.16045975, 2.40110641]),
 array([5.        , 5.        , 5.        , 5.        , 5.        ,
        5.        , 5.        , 5.        , 3.86617078, 2.17656278]),
 array([5.        , 5.        , 5.        , 5.        , 5.        ,
        5.        , 5.        , 4.80268175, 3.4632017 , 1.88004261]),
 array([2.83446901, 2.87060964, 2.84379023, 2.75344956, 2.59769714,
        2.3732737 , 2.07548294, 1.69809329, 1.23320748, 0.6710973 ]),
 array([5.        , 5.        , 5.        , 5.        , 5.        ,
        5.        , 5.        , 5.        , 5.        , 4.45077537]),
 array([4.32255375, 4.38968405, 4.

In [296]:
from pickle import dump
dump(ds, open("mpc.pkl", "wb"))